<a href="https://colab.research.google.com/github/Jan-Novak00/Individual-Software-Project---dynamic-constraint-data-visualization/blob/main/kiwi_demo_MK3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kiwisolver
from kiwisolver import Variable, Constraint, Solver
from PIL import Image, ImageDraw
from IPython.display import display
import random

class VariablePoint2D:
  def __init__(self):
      self.X = Variable("X")
      self.Y = Variable("Y")

class ValuePoint2D:
    def __init__(self, X: int, Y: int):
        self.X = X
        self.Y = Y
    def __str__(self):
        return f"({self.X}, {self.Y})"

class Rectangle:
    def __init__(self, width: Variable, height: int, name: str, color: str | int = "blue"):
        self.height = Variable(f"{name}_height")
        self.width = width
        self.name = name
        self.color = color
        self.leftBottom = VariablePoint2D()
        self.rightTop = VariablePoint2D()

        self.constraints = []

        self._createHeightConstraint(height)
        self._createCornerConstraints()

    def _createHeightConstraint(self, height: int):
        self.constraints.append(self.height == height)

    def _createCornerConstraints(self):
        self.constraints.append(self.leftBottom.X + self.width == self.rightTop.X)
        self.constraints.append(self.leftBottom.Y + self.height == self.rightTop.Y)

    def FixBottomLeftCorner(self, X: int, Y: int):
        self.constraints.append(self.leftBottom.X == X)
        self.constraints.append(self.leftBottom.Y == Y)

    def GetConstraints(self):
        return self.constraints



class BarChart:
  def __init__(self, heights: list[int], width: int, spacing: int):
      self.spacing = Variable("spacing")
      self.width = Variable("width")
      self.constraints = []
      self.rectangles = []

      self._createSpacingConstraint(spacing)
      self._createWidthConstraint(width)
      self._createRectangles(heights)
      self._spaceOutRectangles()

  def _createSpacingConstraint(self, spacing: int):
      self.constraints.append(self.spacing == spacing)

  def _createWidthConstraint(self, width: int):
      self.constraints.append(self.width == width)

  def _createRectangles(self, heights: list[int]):
      for i in range(len(heights)):
          self.rectangles.append(Rectangle(self.width, heights[i], f"rectangle_{i}", random.randint(0, 0xFFFFFF)))

  def _fixFirstRectangle(self, X: int, Y: int):
      self.rectangles[0].FixBottomLeftCorner(X,Y)

  def _spaceOutRectangles(self):
      for i in range(len(self.rectangles)-1):
          self.constraints.append(self.rectangles[i].rightTop.X + self.spacing == self.rectangles[i+1].leftBottom.X)

  def GetValues(self, xCoordinate: int, yCoordinate: int, dumpOutput: bool = False):
      finalConstraints = []
      self._fixFirstRectangle(xCoordinate, yCoordinate)
      finalConstraints += self.constraints
      for rec in self.rectangles:
          finalConstraints += rec.GetConstraints()
          finalConstraints.append(rec.leftBottom.Y == yCoordinate) # move somewhere else?

      solver = Solver()
      for constr in finalConstraints:
          solver.addConstraint(constr)

      solver.updateVariables()

      if dumpOutput:
          solver.dump()

      result = []
      for rec in self.rectangles:
          leftBottom = ValuePoint2D(rec.leftBottom.X.value(), rec.leftBottom.Y.value())
          rightTop = ValuePoint2D(rec.rightTop.X.value(), rec.rightTop.Y.value())
          result.append((leftBottom, rightTop, rec.color))
      return result





In [ ]:
def generateRectangleImage(rectangleInfo: list[tuple[ValuePoint2D, ValuePoint2D, str]], imageWidth: int, imageHeight: int):
    img = Image.new("RGB", (imageWidth, imageHeight), "white")
    draw = ImageDraw.Draw(img)
    for rec in rectangleInfo:
        draw.rectangle([rec[0].X, imageHeight - rec[1].Y, rec[1].X, imageHeight - rec[0].Y], fill=rec[2], outline="black")
    display(img)

    pass

#values = [50, 140, 90]
import numpy as np

values = list(map(int, np.random.poisson(50, 200)))
width = 5
space = 0

barChart = BarChart(values, width, space)

coordinates = barChart.GetValues(0,0,True)

#for c in coordinates:
#    	print(f"(leftBottom = {c[0]}, rightTop = {c[1]}, color = {c[2]})")

generateRectangleImage(coordinates, 1000, 200)